In [1]:
from numba import cuda,jit
import numpy as np
import cv2
import math
from copy import copy
import random
from datetime import datetime

In [2]:
@cuda.jit(device=True)
def compare_pixel(pixela, pixelb):
    c=0
    for i in range(3):
        c+=(pixela[i]-pixelb[i])**2
    return c**0.5

In [3]:
@cuda.jit
def gcompare(imga, imgb, C):
    y, x = cuda.grid(2)
    if y>=imga.shape[0] or x>=imga.shape[1]:
        return
    
    C[y,x]=compare_pixel(imga[y,x], imgb[y,x])

In [4]:
def compare(dimga, dimgb, **params):

    tpb = 16
    bpgy = (dimga.shape[0]-1)//tpb+1
    bpgx = (dimga.shape[1]-1)//tpb+1
    
    C=np.array(np.zeros(imga.shape[0:2]), dtype=np.float32)

    gcompare[(bpgy, bpgx), (tpb, tpb)](np.ascontiguousarray(dimga), np.ascontiguousarray(dimgb), C)

    return 1-np.sum(C)/(dimga.shape[0]*dimga.shape[1]*442)


In [5]:
img=cv2.imread("puzzle.jpg", 1)
imga=img[0:100,0:100]
imgb=img[100:200,100:200]
imgc=imgb+1
imgd=imgb+5
imge=imgb+10
imgf=imgb-1
imgg=imgb-5
imgh=imgb-10


In [6]:
def locate_one_piece(dpiece, solution, **params):
    """Will only receive preprocessed pieces!"""
    max_resemblance=(0, None)
    
    for i in range(solution.dpieces.shape[0]):
        if solution.availability[i]==True:
            resemblance=compare(dpiece, solution.dpieces[i], **params)
            if "debug_mode" in params:
                if params["debug_mode"]==True: print(resemblance)
            if resemblance>max_resemblance[0]:
                max_resemblance=(resemblance, i)
    solution.availability[max_resemblance[1]]=False
    return solution.locations[max_resemblance[1]]


In [7]:
def preprocess_pieces(pieces, solution_pieces, **params):
    if not(isinstance(pieces, np.ndarray) and isinstance(solution_pieces, np.ndarray)):raise TypeError("Wrong object type!")
    if not(len(pieces.shape)==4 and len(solution_pieces.shape)==4): raise Exception("Incorrect array shape!")
    if pieces.shape[0] != solution_pieces.shape[0]: raise Exception("Number of pieces don't match!")
    
    if pieces.shape != solution_pieces.shape:
        print("Piece shape mismatch!")
        pieces=resize(pieces, solution_pieces[0].shape[1:])
    
    return pieces

In [8]:
def resize(pieces, size):
    raise Exception("WRONG SIZE")

In [9]:
def locate_pieces(pieces, solution, **params):
    pieces = preprocess_pieces(pieces, solution.pieces, **params)
    dpieces=cuda.to_device(np.ascontiguousarray(pieces))
    solved_locations=[]
    for i in range(len(solution.locations)):
        location=locate_one_piece(dpieces[i], solution, **params)
        solved_locations.append(location)
    return (pieces, np.array(solved_locations))
        

In [10]:
pieces=[imgb+i for i in range(0,100,5)]
pieces=[imga,imgb]
solutions=(np.stack(pieces), np.arange(20))

In [11]:
class Solution(object):
    def __init__(self, path, dims):
        self.pieces=img_split(path, dims)  
        self.dpieces=cuda.to_device(np.ascontiguousarray(self.pieces))
        self.locations=np.array([(i,j) for i in range(dims[1]) for j in range(dims[0])])
        self.availability=[True]*dims[0]*dims[1]

In [12]:
def brg_to_rgb(image):
    b,g,r=np.split(image, 3, axis=2)
    return np.concatenate((r,g,b), axis=2)

def img_split(image_path, dims, **params):
    assert type(dims)==tuple
    image=brg_to_rgb(cv2.imread(image_path, 1))

    pieces=[]
    height=image.shape[0]/dims[0]
    width=image.shape[1]/dims[1]
    for y_split in range(dims[0]):
        for x_split in range(dims[1]):
            x_start=int(x_split*width)
            x_end=x_start+int(width)
            y_start=int(y_split*height)
            y_end=y_start+int(height)

            pieces.append(np.array(image[y_start: y_end, x_start: x_end]))
    return np.array(pieces)

In [13]:
%timeit sol=Solution("puzzle.jpg", (5,5)); locate_pieces(img_split("puzzle.jpg", (5,5)), sol)[1]

872 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
def pool(image, pooling):
    pass

In [15]:
from img_recog_proto import openimg, writeimg